# Qdrant Experiment Example

## Installations

In [1]:
# !pip install --quiet --force-reinstall prompttools
# !pip install qdrant-client

Qdrant offers three deployment modes: in-memory/on-disk built-in into Python client, on-premise with Docker containers and Qdrant Cloud. For the purposes of the demo it's easiest to spin up a Docker container with the following command:

In [2]:
# !docker run -p "6333:6333" -p "6334:6334" qdrant/qdrant:v1.4.0

## Run an experiment

Collection configuration plays a crucial role in balancing speed vs accuracy. HNSW parameters might be tuned to control how exact the search operations will be, but on the other hand that may result in increased latency. There are also some other parameters, such as embeddings or distance function you may want to test before starting a project.

Embedding function is a key component for semantic search. For the purposes of that experiment we are going to use `all-MiniLM-L6-v2` from SentenceTransformers.

In [3]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def embedding_function(text: str):
    return model.encode(text).tolist()

/home/kacper/.virtualenvs/prompttools/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Some of the parameters are going to be fixed during the whole experiment. We are not going to change the Qdrant client, embedding function, vector size, document and queries. We can define them as a first dictionary.

In [4]:
from qdrant_client import QdrantClient


frozen_parameters = {
    "client": QdrantClient("http://localhost:6333"),
    "collection_name": "test_collection",
    "embedding_fn": embedding_function,
    "vector_size": 384,  # We need to know the dimensionality of the embedding
    "documents": ["test document 1", "test document 2"],
    "queries": ["test query 1", "test query 2"],
}

Our experiment will check how the system behaves under different configurations. We can define both collection and query parameters:

In [5]:
test_parameters = {
    "collection_params": {
        "vectors_config__distance": ["Cosine", "Euclid", "Dot"],
        "hnsw_config__m": [16, 32, 64, 128],
    },
    "query_params": {
        "search_params__hnsw_ef": [1, 16, 32, 64, 128],
        "search_params__exact": [True, False],
    },
}

Finally, we are able to launch the experiment.

In [6]:
from prompttools.experiment import QdrantExperiment


experiment = QdrantExperiment.initialize(test_parameters=test_parameters, frozen_parameters=frozen_parameters)
experiment.run()

The results of the experiment might be used further to calculate some metrics and determine the best configuration of the engine in a particular scenario.

In [7]:
experiment.visualize()

,search_params,query_vector,response,latency
0,"{'hnsw_ef': 1, 'exact': True}","[0.024773715063929558, 0.04144531488418579, 0.01278286799788475, 0.09144901484251022, -0.08428235352039337, 0.026642095297574997, 0.14001256227493286, 0.037845369428396225, -0.036010514944791794, -0.035383205860853195, 0.07331386208534241, -0.09981919080018997, 0.07208795100450516, -0.08177464455366135, -0.03372645005583763, -0.026198092848062515, 0.007823744788765907, -0.05943206325173378, -0.045515742152929306, 0.01116162072867155, -0.036530762910842896, -0.04401981085538864, -0.0475635901093483, 0.070327989757061, 0.019287584349513054, -0.018562251701951027, 0.021239709109067917, 0.014735235832631588, 0.03941147401928902, -0.057817116379737854, -0.019634393975138664, 0.043356701731681824, -0.03757508099079132, 0.06539657711982727, 0.10788384079933167, -0.09214196354150772, 0.0677819475531578, -0.0006760165560990572, -0.01706380769610405, 5.571492147282697e-05, -0.005168926436454058, -0.11285313218832016, 0.06314507126808167, -0.04196688160300255, -0.014073682017624378, 0.047889478504657745, -0.02251909300684929, 0.13618265092372894, -0.040116459131240845, -0.04491341859102249, -0.11577481031417847, -0.017022281885147095, -0.09723599255084991, -0.02146202139556408, -0.021104147657752037, -0.0209374837577343, -0.08296425640583038, 0.007878098636865616, -0.0194426067173481, 0.04828505963087082, -0.009565607644617558, -0.0198497474193573, 0.08935166150331497, 0.027390005066990852, 0.05083496868610382, -0.08648493140935898, 0.02043042704463005, -0.05061521753668785, 0.0643402561545372, -0.0330120213329792, -0.04423206299543381, 0.001910282182507217, -0.03994475677609444, 0.045240066945552826, -0.006637560669332743, 0.01001442689448595, -0.05905541777610779, -0.05051897466182709, 0.04575075954198837, -0.05852947756648064, -0.04943345859646797, -0.07612475752830505, -0.070700503885746, 0.016476262360811234, 0.022484982386231422, 0.05507957190275192, 0.11701894551515579, -0.0023378280457109213, -0.03449930250644684, -0.044528309255838394, 0.0598871186375618, 0.05605398491024971, -0.04443911090493202, 0.042640648782253265, 0.02585383877158165, 0.017724832519888878, 0.03406650945544243, -0.023438140749931335, 0.0613379031419754, 0.11572054773569107, ...]","[test document 1, test document 2]",0.028570
1,"{'hnsw_ef': 1, 'exact': True}","[0.04173979535698891, 0.05546772852540016, 0.020573439076542854, 0.08281106501817703, -0.08270967751741409, 0.023233482614159584, 0.11520404368638992, 0.04725329205393791, -0.03447265177965164, -0.040253203362226486, 0.10197387635707855, -0.10148394852876663, 0.059999171644449234, -0.08607508987188339, -0.030827024951577187, -0.014220409095287323, 0.014857304282486439, -0.037770576775074005, -0.03734028711915016, -0.010152825154364109, -0.005581939127296209, -0.038822367787361145, -0.03395431861281395, 0.049912165850400925, 0.004501402378082275, 0.012198868207633495, 0.009263121522963047, 0.021756915375590324, 0.04381023347377777, -0.05678417533636093, -0.0228099524974823, 0.0496208630502224, -0.07200998812913895, 0.05564706027507782, 0.0998462662100792, -0.10447440296411514, 0.0801955834031105, 0.005559707060456276, -0.0032220862340182066, -0.01756117306649685, -0.01011226698756218, -0.12116091698408127, 0.058426838368177414, -0.033617377281188965, 0.0041591632179915905, 0.06505881249904633, -0.010240594856441021, 0.11785278469324112, -0.034987181425094604, -0.035445909947156906, -0.12772171199321747, -0.01835581660270691, -0.09354864805936813, -0.0261138454079628, -0.016325369477272034, 0.01150042936205864, -0.08788514882326126, 0.01320055965334177, -0.0148471025750041, 0.0443340465426445, -0.004045989830046892, -0.014519786462187767, 0.0674666166305542, 0.03183493763208389, 0.04295126348733902, -0.09864385426044464, 0.015303893014788628, -0.0419088713824749, 0.06560134887695312, -0.042264118790626526, -0.02623181790113449, 0.018841687589883804, -0.03793366253376007, 0.0456632599234581, 0.009519274346530437, 0.007414438296